In [1]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import os
from datetime import datetime
from dateutil.parser import parse

In [2]:
# working directory
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Arizona/WaterAllocation/RawInputData"
os.chdir(workingDir)

In [3]:
######## WaDE columns

#the followwing fields have difference between the table here (edited by DPL) and that on the 
# schema website
#http://schema.westernstateswater.org/tables/Input_AllocationAmounts_fact.html
"""
BeneficialUseCategory, PrimaryUseCategory, AllocationTimeframeStart, AllocationTimeframeEnd, " "
BeneficialUseCategoryCV, PrimaryUseCategoryCV, TimeframeStartDate,	TimeframeEndDate,	Geometry
"""
# UUIDs: Add UUIDs for all dim tables
# OrganizationUUID, SiteUUID, VariableSpecificUUID, WaterSourceUUID, MethodUUID
columns = ["OrganizationUUID", "SiteUUID", "VariableSpecificUUID", "WaterSourceUUID","MethodUUID",
           "PrimaryUseCategory",
           "BeneficialUseCategory", "AllocationNativeID", "AllocationTypeCV", "AllocationOwner",
           "AllocationApplicationDate", "AllocationPriorityDate", "AllocationLegalStatusCV", 
           "AllocationCropDutyAmount",
           "AllocationExpirationDate", "AllocationChangeApplicationIndicator", 
           "LegacyAllocationIDs", "AllocationBasisCV", "AllocationTimeframeStart",
           "AllocationTimeframeEnd", "AllocationAmount", "AllocationMaximum", "PopulationServed",
           "PowerType","GeneratedPowerCapacityMW", "IrrigatedAcreage",
           "AllocationCommunityWaterSupplySystem", "AllocationSDWISIdentifierCV",
           "AllocationAssociatedWithdrawalSiteIDs", "AllocationAssociatedConsumptiveUseSiteIDs",
           "WaterAllocationNativeURL", "CustomerTypeCV", "IrrigationMethodCV", "CropTypeCV",
           "CommunityWaterSupplySystem", "DataPublicationDate", "DataPublicationDOI"]

dtypesx = [''] #here we could theoretically specify data types for each column name,
              #but we didn't need to do that

In [4]:
### target dataFrame

# TODO: assumes dtypes inferred from CO file
outdf100=pd.DataFrame(columns=columns)

In [8]:
# Input files
fileInput1 = "Well_Registry_Wells55_old.csv" 
fileInput2 = "GWSI_Sites_old.csv"

#output: water allocation
out_alloc = "waterallocations.csv"    #output

In [9]:
print("Reading inputs...")

# Read Inputs 
# 
df200 = pd.read_csv(fileInput1,encoding = "ISO-8859-1") #, or alternatively encoding = "utf-8"
print (len(df200.index))
df200.drop_duplicates(inplace=True)
print(len(df200))
#df200

# columns of GWSI_Sites
# "X", "Y", "OBJECTID", "SITE_ID", "LOCAL_ID", "REG_ID", "WELL_TYPE", "DD_LAT", "DD_LONG",
# "LAT_NAD27", "LONG_NAD27", "WELL_ALT", "WATER_USE", "WELL_DEPTH", "CASE_DIAM", "DRILL_DATE", 
# "WL_COUNT", "LASTWLDATE", "WL_DTW", "WL_ELEV", "SOURCE", "IDXBK

cols_GWSI = ["SITE_ID", "LOCAL_ID", "REG_ID", "WELL_TYPE", "DD_LAT", "DD_LONG",
      "WATER_USE",  "DRILL_DATE", "LASTWLDATE", "WL_DTW", "SOURCE", "IDXBK"]

df300 = pd.read_csv(fileInput2,encoding = "ISO-8859-1", usecols = cols_GWSI) #, or alternatively encoding = "utf-8"
print (len(df300.index))
df200.drop_duplicates(inplace=True)
print(len(df300))
#df300

Reading inputs...


C:\Users\rjame\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


216408
216408
45460
45460


In [10]:
print("Join tables...")

df100=pd.merge(df200, df300, left_on='REGISTRY_ID', right_on='REG_ID', how='inner') #

#print (len(df100.index))

#df100 = df100.head(10000) #only runs first 100 lines for testing.

df100 = df100.replace(np.nan, '')

df100
#df100.head(5)

Join tables...


,ï»¿X,Y,OBJECTID,PROGRAM,REGISTRY_ID,OWNER_NAME,RGR_PUMP_DATA,WELLTYPE,WELL_TYPE_GROUP,DLIC_NUM,...,REG_ID,WELL_TYPE,DD_LAT,DD_LONG,WATER_USE,DRILL_DATE,LASTWLDATE,WL_DTW,SOURCE,IDXBK
0,-112.498143,33.784977,2,55,60001,"SFI GRAND VISTA, LLC",YES,NON-EXEMPT - NON-SERVICE,NON-EXEMPT,611,...,60001.0,INDEX,33.785056,-112.497806,INDUSTRIAL,7/2/1984,12/7/2018,489,,BK02
1,-110.231423,31.346352,10,55,84460,RICHARD RICHARDS,NO,EXEMPT,EXEMPT,152,...,84460.0,GWSI,31.347036,-110.230244,UNUSED,6/16/1980,6/16/1980,150,,
2,-114.574586,34.863445,15,55,84630,JOHN CLAYPOOL,NO,NON-EXEMPT,NON-EXEMPT,157,...,84630.0,GWSI,34.863611,-114.573889,DOMESTIC,6/20/1980,6/20/1980,18,,
3,-114.576929,34.890564,16,55,84633,HUBSON,NO,NON-EXEMPT,NON-EXEMPT,157,...,84633.0,GWSI,34.890833,-114.575833,DOMESTIC,6/18/1980,6/18/1980,18,,
4,-111.498588,32.604010,20,55,84660,"KEP INC,",NO,NON-EXEMPT - REPLACEMENT WELL IN NEW LOCATION,NON-EXEMPT,1,...,84660.0,GWSI,32.604056,-111.497833,UNUSED,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24285,-112.244560,34.674876,215787,55,923132,BRYCE ENTERPRISES,NO,EXEMPT,EXEMPT,331,...,923132.0,GWSI,34.675333,-112.243111,DOMESTIC,7/22/2019,7/22/2019,100,,
24286,-112.242366,34.674871,215788,55,923133,BRYCE ENTERPRISES,NO,EXEMPT,EXEMPT,331,...,923133.0,GWSI,34.675334,-112.242332,UNUSED,7/22/2019,,,,
24287,-112.244560,34.674876,215789,55,923134,DNC DEVELOPMENT LLC,NO,EXEMPT,EXEMPT,331,...,923134.0,GWSI,34.675334,-112.244781,UNUSED,7/11/2019,,,,
24288,-112.244560,34.674876,215790,55,923135,DNC DEVELOPMENT LLC,NO,EXEMPT,EXEMPT,331,...,923135.0,GWSI,34.675333,-112.243944,UNUSED,7/23/2019,,,,


In [11]:
# use only unique water rights that may have multiple sites/pds
print("Dropping duplicates...")

df100.drop_duplicates(subset = ['REGISTRY_ID'], inplace=True)   #
df100 = df100.reset_index(drop=True)

print (len(df100.index))

df100

Dropping duplicates...
24026


,ï»¿X,Y,OBJECTID,PROGRAM,REGISTRY_ID,OWNER_NAME,RGR_PUMP_DATA,WELLTYPE,WELL_TYPE_GROUP,DLIC_NUM,...,REG_ID,WELL_TYPE,DD_LAT,DD_LONG,WATER_USE,DRILL_DATE,LASTWLDATE,WL_DTW,SOURCE,IDXBK
0,-112.498143,33.784977,2,55,60001,"SFI GRAND VISTA, LLC",YES,NON-EXEMPT - NON-SERVICE,NON-EXEMPT,611,...,60001.0,INDEX,33.785056,-112.497806,INDUSTRIAL,7/2/1984,12/7/2018,489,,BK02
1,-110.231423,31.346352,10,55,84460,RICHARD RICHARDS,NO,EXEMPT,EXEMPT,152,...,84460.0,GWSI,31.347036,-110.230244,UNUSED,6/16/1980,6/16/1980,150,,
2,-114.574586,34.863445,15,55,84630,JOHN CLAYPOOL,NO,NON-EXEMPT,NON-EXEMPT,157,...,84630.0,GWSI,34.863611,-114.573889,DOMESTIC,6/20/1980,6/20/1980,18,,
3,-114.576929,34.890564,16,55,84633,HUBSON,NO,NON-EXEMPT,NON-EXEMPT,157,...,84633.0,GWSI,34.890833,-114.575833,DOMESTIC,6/18/1980,6/18/1980,18,,
4,-111.498588,32.604010,20,55,84660,"KEP INC,",NO,NON-EXEMPT - REPLACEMENT WELL IN NEW LOCATION,NON-EXEMPT,1,...,84660.0,GWSI,32.604056,-111.497833,UNUSED,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24021,-112.244560,34.674876,215787,55,923132,BRYCE ENTERPRISES,NO,EXEMPT,EXEMPT,331,...,923132.0,GWSI,34.675333,-112.243111,DOMESTIC,7/22/2019,7/22/2019,100,,
24022,-112.242366,34.674871,215788,55,923133,BRYCE ENTERPRISES,NO,EXEMPT,EXEMPT,331,...,923133.0,GWSI,34.675334,-112.242332,UNUSED,7/22/2019,,,,
24023,-112.244560,34.674876,215789,55,923134,DNC DEVELOPMENT LLC,NO,EXEMPT,EXEMPT,331,...,923134.0,GWSI,34.675334,-112.244781,UNUSED,7/11/2019,,,,
24024,-112.244560,34.674876,215790,55,923135,DNC DEVELOPMENT LLC,NO,EXEMPT,EXEMPT,331,...,923135.0,GWSI,34.675333,-112.243944,UNUSED,7/23/2019,,,,


In [12]:
print("Adding SiteUUID...")

df100 = df100.assign(SiteUUID='')  #add new column and make is nan

#Permit Number
df100['SiteUUID'] = df100.apply(lambda row: '_'.join(['AZ', str(row["SITE_ID"])]), axis=1)

df100

Adding SiteUUID...


,ï»¿X,Y,OBJECTID,PROGRAM,REGISTRY_ID,OWNER_NAME,RGR_PUMP_DATA,WELLTYPE,WELL_TYPE_GROUP,DLIC_NUM,...,WELL_TYPE,DD_LAT,DD_LONG,WATER_USE,DRILL_DATE,LASTWLDATE,WL_DTW,SOURCE,IDXBK,SiteUUID
0,-112.498143,33.784977,2,55,60001,"SFI GRAND VISTA, LLC",YES,NON-EXEMPT - NON-SERVICE,NON-EXEMPT,611,...,INDEX,33.785056,-112.497806,INDUSTRIAL,7/2/1984,12/7/2018,489,,BK02,AZ_334708112295301
1,-110.231423,31.346352,10,55,84460,RICHARD RICHARDS,NO,EXEMPT,EXEMPT,152,...,GWSI,31.347036,-110.230244,UNUSED,6/16/1980,6/16/1980,150,,,AZ_312049110134902
2,-114.574586,34.863445,15,55,84630,JOHN CLAYPOOL,NO,NON-EXEMPT,NON-EXEMPT,157,...,GWSI,34.863611,-114.573889,DOMESTIC,6/20/1980,6/20/1980,18,,,AZ_345149114342601
3,-114.576929,34.890564,16,55,84633,HUBSON,NO,NON-EXEMPT,NON-EXEMPT,157,...,GWSI,34.890833,-114.575833,DOMESTIC,6/18/1980,6/18/1980,18,,,AZ_345327114343301
4,-111.498588,32.604010,20,55,84660,"KEP INC,",NO,NON-EXEMPT - REPLACEMENT WELL IN NEW LOCATION,NON-EXEMPT,1,...,GWSI,32.604056,-111.497833,UNUSED,,,,,,AZ_323616111295201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24021,-112.244560,34.674876,215787,55,923132,BRYCE ENTERPRISES,NO,EXEMPT,EXEMPT,331,...,GWSI,34.675333,-112.243111,DOMESTIC,7/22/2019,7/22/2019,100,,,AZ_344031112143801
24022,-112.242366,34.674871,215788,55,923133,BRYCE ENTERPRISES,NO,EXEMPT,EXEMPT,331,...,GWSI,34.675334,-112.242332,UNUSED,7/22/2019,,,,,AZ_344031112143501
24023,-112.244560,34.674876,215789,55,923134,DNC DEVELOPMENT LLC,NO,EXEMPT,EXEMPT,331,...,GWSI,34.675334,-112.244781,UNUSED,7/11/2019,,,,,AZ_344031112144401
24024,-112.244560,34.674876,215790,55,923135,DNC DEVELOPMENT LLC,NO,EXEMPT,EXEMPT,331,...,GWSI,34.675333,-112.243944,UNUSED,7/23/2019,,,,,AZ_344031112144101


In [13]:
print("Allocation priority date...")

# input format 1900-01-01T00:00:00.000Z 
def formatDateString(inString):
    #print(inString)
    try:
        if inString == '' or pd.isnull(inString):
            valndf = ''
        else:
            valD = datetime.strptime(inString, '%Y-%m-%dT00:00:00.000Z')
            #print(valD)
            valnDd = valD.date()
            #print(valnDd)
            valndf = valnDd.strftime('%m/%d/%Y')
            #print('date:', valndf)
    except:
        valndf = ''

    return valndf

df100 = df100.assign(AllocationPriorityDate='')

df100['AllocationPriorityDate'] = df100.apply(lambda row: 
                                        formatDateString(row['APPLICATION_DATE']), axis=1)

df100

Allocation priority date...


,ï»¿X,Y,OBJECTID,PROGRAM,REGISTRY_ID,OWNER_NAME,RGR_PUMP_DATA,WELLTYPE,WELL_TYPE_GROUP,DLIC_NUM,...,DD_LAT,DD_LONG,WATER_USE,DRILL_DATE,LASTWLDATE,WL_DTW,SOURCE,IDXBK,SiteUUID,AllocationPriorityDate
0,-112.498143,33.784977,2,55,60001,"SFI GRAND VISTA, LLC",YES,NON-EXEMPT - NON-SERVICE,NON-EXEMPT,611,...,33.785056,-112.497806,INDUSTRIAL,7/2/1984,12/7/2018,489,,BK02,AZ_334708112295301,02/25/1985
1,-110.231423,31.346352,10,55,84460,RICHARD RICHARDS,NO,EXEMPT,EXEMPT,152,...,31.347036,-110.230244,UNUSED,6/16/1980,6/16/1980,150,,,AZ_312049110134902,06/12/1980
2,-114.574586,34.863445,15,55,84630,JOHN CLAYPOOL,NO,NON-EXEMPT,NON-EXEMPT,157,...,34.863611,-114.573889,DOMESTIC,6/20/1980,6/20/1980,18,,,AZ_345149114342601,07/06/1980
3,-114.576929,34.890564,16,55,84633,HUBSON,NO,NON-EXEMPT,NON-EXEMPT,157,...,34.890833,-114.575833,DOMESTIC,6/18/1980,6/18/1980,18,,,AZ_345327114343301,06/06/1980
4,-111.498588,32.604010,20,55,84660,"KEP INC,",NO,NON-EXEMPT - REPLACEMENT WELL IN NEW LOCATION,NON-EXEMPT,1,...,32.604056,-111.497833,UNUSED,,,,,,AZ_323616111295201,08/21/1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24021,-112.244560,34.674876,215787,55,923132,BRYCE ENTERPRISES,NO,EXEMPT,EXEMPT,331,...,34.675333,-112.243111,DOMESTIC,7/22/2019,7/22/2019,100,,,AZ_344031112143801,06/28/2019
24022,-112.242366,34.674871,215788,55,923133,BRYCE ENTERPRISES,NO,EXEMPT,EXEMPT,331,...,34.675334,-112.242332,UNUSED,7/22/2019,,,,,AZ_344031112143501,06/28/2019
24023,-112.244560,34.674876,215789,55,923134,DNC DEVELOPMENT LLC,NO,EXEMPT,EXEMPT,331,...,34.675334,-112.244781,UNUSED,7/11/2019,,,,,AZ_344031112144401,06/28/2019
24024,-112.244560,34.674876,215790,55,923135,DNC DEVELOPMENT LLC,NO,EXEMPT,EXEMPT,331,...,34.675333,-112.243944,UNUSED,7/23/2019,,,,,AZ_344031112144101,07/01/2019


In [14]:
df100.to_csv('df100_old.csv', index=False)  # The output

In [ ]:
print("Copying all columns...")
#
destCols=["SiteUUID", 
          #"WaterSourceUUID", 
          "AllocationNativeID", 
          #"AllocationLegalStatusCV", 
          "BeneficialUseCategory", 
          "AllocationOwner", 
          "AllocationTypeCV", 
          #"AllocationApplicationDate", 
          "AllocationPriorityDate",
          "AllocationAmount", 
          "AllocationMaximum", 
          #"IrrigatedAcreage",
          #"AllocationCropDutyAmount", "AllocationExpirationDate", 
          #"TimeframeStart", "TimeframeEnd",
          #"WaterAllocationNativeURL"
         ]
#
srsCols=["SiteUUID", 
         #"WaterSourceUUID", 
          "REGISTRY_ID", 
         #"WaRecProcessStatusTypeCode",
          "WATER_USE", 
          "OWNER_NAME",
          "WELLTYPE", 
          #"AllocationApplicationDate",
          "AllocationPriorityDate", 
          "PUMPRATE",
          "TESTEDRATE",
          #"IrrigatedAreaQuantity",
          #"IRRIGATION_DEPLETION", "DATE_TERMINATED",
          #"TimeframeStart", "TimeframeEnd",
          #"NRIS_LINK"
         ]

outdf100[destCols] = df100[srsCols]

outdf100

In [ ]:
# hard coded
print("Hard coded...")
#hard coded
outdf100.OrganizationUUID = "ADWR"
outdf100.VariableSpecificUUID = "ADWR Allocation all"
outdf100.MethodUUID = "ADWR-Water Rights"
outdf100.AllocationBasisCV = "Unknown"
outdf100.WaterSourceUUID = "AZ_1"
# check this later
outdf100.PrimaryUseCategory = "Irrigation"
#
outdf100.TimeframeStart = "01/01"
outdf100.TimeframeEnd = "12/31"

#
outdf100.DataPublicationDate = datetime.now().strftime('%m/%d/%Y')    #"10/31/2019" # edit this to the code run date

outdf100

In [ ]:
print("Droping null allocations...")
# if both Allocation amount and Allocation maximum are empty drop row and save it to a Allocations_missing.csv
outdf100 = outdf100.replace(np.nan, '') #replace blank strings by NaN,
outdf100purge = outdf100.loc[(outdf100["AllocationAmount"] == '') & (outdf100["AllocationMaximum"] == '')]
if len(outdf100purge.index) > 0:
    outdf100purge.to_csv('waterallocations_missing.csv')    #index=False,
    dropIndex = outdf100.loc[(outdf100["AllocationAmount"] == '') & (outdf100["AllocationMaximum"] == '')].index
    outdf100 = outdf100.drop(dropIndex)
    outdf100 = outdf100.reset_index(drop=True)

outdf100
#outdf100purge

In [ ]:
print("Droping null SiteUUIDs...")
outdf100nullID = outdf100.loc[outdf100["SiteUUID"] == '']
if len(outdf100nullID.index) > 0:
    dropIndex = outdf100.loc[outdf100["SiteUUID"] == ''].index
    outdf100 = outdf100.drop(dropIndex)
    outdf100 = outdf100.reset_index(drop=True)

outdf100

In [ ]:
print("Droping null Priority date...")
outdf100nullPR = outdf100.loc[outdf100["AllocationPriorityDate"] == '']
if len(outdf100nullPR.index) > 0:
    dropIndex = outdf100.loc[outdf100["AllocationPriorityDate"] == ''].index
    outdf100 = outdf100.drop(dropIndex)
    outdf100 = outdf100.reset_index(drop=True)

outdf100
#outdf100nullPR

In [ ]:
print("Droping null WaterSourceUUID ...")
outdf100nullPR = outdf100.loc[outdf100["WaterSourceUUID"] == '']
if len(outdf100nullPR.index) > 0:
    dropIndex = outdf100.loc[outdf100["WaterSourceUUID"] == ''].index
    outdf100 = outdf100.drop(dropIndex)
    outdf100 = outdf100.reset_index(drop=True)
outdf100

In [ ]:
print("Droping duplicates...")
#drop duplicate rows; just make sure
outdf100Duplicated=outdf100.loc[outdf100.duplicated()]
if len(outdf100Duplicated.index) > 0:
    outdf100Duplicated.to_csv("waterallocations_duplicaterows.csv")  # index=False,
    outdf100.drop_duplicates(inplace=True)   #
    outdf100 = outdf100.reset_index(drop=True)

outdf100

In [ ]:
print("Checking required is not null...")
# check if any cell of these columns is null
requiredCols = ["OrganizationUUID", "VariableSpecificUUID", "WaterSourceUUID", 
                "MethodUUID", "AllocationPriorityDate"] #SiteUUID
# outdf100_nullMand = outdf100.loc[outdf100.isnull().any(axis=1)] --for all cols
# outdf100_nullMand = outdf100.loc[outdf100[requiredCols].isnull().any(axis=1)]
#(outdf100["SiteUUID"].isnull()) |
outdf100_nullMand = outdf100.loc[(outdf100["OrganizationUUID"] == '') |
                                (outdf100["VariableSpecificUUID"] == '') |
                                (outdf100["WaterSourceUUID"] == '') |
                                (outdf100["MethodUUID"] == '') |
                                (outdf100["AllocationPriorityDate"] == '')]
#outdf100_nullMand = outdf100.loc[[False | (outdf100[varName].isnull()) for varName in requiredCols]]
if(len(outdf100_nullMand.index) > 0):
    outdf100_nullMand.to_csv('waterallocations_mandatoryFieldMissing.csv')  # index=False,
#ToDO: purge these cells if there is any missing? #For now left to be inspected
#outdf100_nullMand

In [ ]:
print("Writing outputs...")
#write out
outdf100.to_csv(out_alloc, index=False, encoding = "utf-8")

print("Done Water Allocation")

### Do not run the following with the rest of the code  (it is for inspection)

In [ ]:
##### Do not run the following with the rest of the code  (it is for inspection)
print("Long site ids...")

#output: water allocation
in_alloc = "waterallocations_long.csv"    #output
# ground water
outdf100 = pd.read_csv(in_alloc, encoding = "ISO-8859-1") #, or alternatively encoding = "utf-8"
print (len(outdf100.index))

outdf100Long = outdf100[outdf100['SiteUUID'].apply(lambda x: len(x) > 500)]
if len(outdf100Long.index) > 0:
    print("There are rows with too long siteids")
    outdf100Long.to_csv("waterallocations_longsiteid.csv")  # index=False,
    dropIndex = outdf100[outdf100['SiteUUID'].apply(lambda x: len(x) > 500)].index
    outdf100 = outdf100.drop(dropIndex)   #
    outdf100 = outdf100.reset_index(drop=True)
#outdf100

outdf100Long

print("Writing outputs...")
#write out
#output: water allocation
out_alloc = "waterallocations.csv"    #output
outdf100.to_csv(out_alloc, index=False, encoding = "utf-8")

print("Done Water Allocation")